### Test spaCy

In [20]:
import spacy

nlp = spacy.load("fr_core_news_lg")

text = """Nous avons un rendez-vous demain à 18h. 
La réunion aura lieu lundi prochain à 10:30.
Le projet démarre le 15/08/2025 et finit dans 2 semaines.
Je suis arrivé il y a 3 jours et je repars vendredi matin. Je retourne a Paris et a angers"""

doc = nlp(text)

for ent in doc.ents:
    if ent.label_ == "LOC":
        print(f"{ent.text} -> {ent.label_}")


Paris -> LOC
angers -> LOC


In [1]:
import spacy
import dateparser
import datetime

nlp = spacy.load("fr_core_news_lg")

text = """Nous avons un rendez-vous demain à 18h. 
La réunion aura lieu lundi prochain à 10:30.
Le projet démarre le 15/08/2025 et finit dans 2 semaines.
Je suis arrivé il y a 3 jours et je repars vendredi matin. Je retourne à Paris et à Angers."""

doc = nlp(text)

dates_found = []
locations_found = []

for ent in doc.ents:
    if ent.label_ == "LOC":
        locations_found.append(ent.text)
    elif ent.label_ == "DATE":
        parsed_date = dateparser.parse(ent.text, settings={'PREFER_DATES_FROM': 'future'})
        if parsed_date:
            dates_found.append(parsed_date)

# Afficher les résultats
print("📅 Dates trouvées et converties :")
for date in dates_found:
    print(f" - {date}")

print("\n🌍 Localisations trouvées :")
for location in locations_found:
    print(f" - {location}")


📅 Dates trouvées et converties :

🌍 Localisations trouvées :
 - Paris
 - Angers


test Date parser

In [21]:
import re
import dateparser

# Texte contenant plusieurs dates et expressions temporelles
text = """
Nous avons un rendez-vous demain à 18h. 
La réunion aura lieu lundi prochain à 10:30.
Le projet démarre le 15/08/2025 et finit dans 2 semaines.
Je suis arrivé il y a 3 jours et je repars vendredi matin.
"""


# Regex pour capturer différentes formes de dates et heures
pattern = r"""
    \b(?:\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{4}-\d{2}-\d{2})\b |  # Dates classiques : 10/02/2024, 2024-05-10
    \b(?:\d{1,2}h(?:\d{2})?)\b |  # Heures : 10h, 14:30
    \b(?:demain|après-demain|hier|aujourd'hui|minuit|midi)\b |  # Expressions directes
    \b(?:dans|il y a)\s\d+\s(?:jours?|semaines?|mois?|années?)\b |  # Relatif : dans 3 jours, il y a 2 semaines
    \b(?:lundi|mardi|mercredi|jeudi|vendredi|samedi|dimanche)\b |  # Jours de la semaine
    \b(?:janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)\b  # Mois
"""

# Extraction des dates et expressions temporelles
matches = re.findall(pattern, text, re.IGNORECASE | re.VERBOSE)

# Suppression des espaces vides et stockage dans un tableau
dates_extracted = [match.strip() for match in matches if match.strip()]

# Affichage des résultats
print("Dates détectées :", dates_extracted)

# Parsing des dates avec dateparser
parsed_dates = [dateparser.parse(date) for date in dates_extracted]

# Affichage des dates converties
for original, parsed in zip(dates_extracted, parsed_dates):
    print(f"{original} -> {parsed}")


Dates détectées : ['demain', '18h', 'lundi', '15/08/2025', 'dans 2 semaines', 'vendredi']
demain -> 2025-02-20 16:25:50.033370
18h -> 2025-02-18 22:25:50.034368
lundi -> 2025-02-17 00:00:00
15/08/2025 -> 2025-08-15 00:00:00
dans 2 semaines -> 2025-03-05 16:25:50.056549
vendredi -> 2025-02-14 00:00:00


### Test geopy

In [22]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="vocal_wheather")

location = geolocator.geocode("angers")

print(location.address)

print((location.latitude, location.longitude))

print(location.raw)


geo_latitude = location.latitude

geo_longitude = location.longitude


Angers, Maine-et-Loire, Pays de la Loire, France métropolitaine, France
(47.4739884, -0.5515588)
{'place_id': 266863829, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 178351, 'lat': '47.4739884', 'lon': '-0.5515588', 'class': 'boundary', 'type': 'administrative', 'place_rank': 16, 'importance': 0.6548475958239229, 'addresstype': 'city', 'name': 'Angers', 'display_name': 'Angers, Maine-et-Loire, Pays de la Loire, France métropolitaine, France', 'boundingbox': ['47.4373546', '47.5262993', '-0.6176931', '-0.5082737']}


### Test open-meteo

In [23]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": geo_latitude,
	"longitude": geo_longitude,
	"current": ["temperature_2m", "is_day", "weather_code"],
	"hourly": ["temperature_2m", "weather_code", "is_day"],
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min"],
	"timezone": "Europe/Berlin",
	"forecast_hours": 12,
	"models": "meteofrance_seamless"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


# Current values. The order of variables needs to be the same as requested.
current = response.Current()

current_temperature_2m = current.Variables(0).Value()

current_is_day = current.Variables(1).Value()

current_weather_code = current.Variables(2).Value()

print(f"Current time {current.Time()}")

print(f"Current temperature_2m {current_temperature_2m}")
print(f"Current is_day {current_is_day}")
print(f"Current weather_code {current_weather_code}")
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(1).ValuesAsNumpy()
hourly_is_day = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["weather_code"] = hourly_weather_code
hourly_data["is_day"] = hourly_is_day

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min



daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)



Coordinates 47.470001220703125°N -0.5500001907348633°E
Elevation 28.0 m asl
Timezone b'Europe/Berlin' b'GMT+1'
Timezone difference to GMT+0 3600 s
Current time 1739978100
Current temperature_2m 15.61050033569336
Current is_day 1.0
Current weather_code 3.0
                        date  temperature_2m  weather_code  is_day
0  2025-02-19 15:00:00+00:00       15.560500           3.0     1.0
1  2025-02-19 16:00:00+00:00       15.360500           1.0     1.0
2  2025-02-19 17:00:00+00:00       14.860500           3.0     1.0
3  2025-02-19 18:00:00+00:00       14.310500           3.0     0.0
4  2025-02-19 19:00:00+00:00       12.110500           3.0     0.0
5  2025-02-19 20:00:00+00:00       10.610500           3.0     0.0
6  2025-02-19 21:00:00+00:00        9.910501           3.0     0.0
7  2025-02-19 22:00:00+00:00        8.960501           3.0     0.0
8  2025-02-19 23:00:00+00:00        8.610500           3.0     0.0
9  2025-02-20 00:00:00+00:00        8.660501           3.0     0.0
10 2025

In [24]:
current_dict = {
    'time' : [current.Time()],
    'temperature' : [current_temperature_2m],
    'is_day' : [current_is_day],
    'weather_code' : [current_weather_code]
}
current_df = pd.DataFrame(current_dict)
current_df

,time,temperature,is_day,weather_code
0,1739978100,15.6105,1.0,3.0


In [26]:
hourly_dict = {
    'date' : hourly_data['date'],
    'temperature' : hourly_temperature_2m,
    'weather_code' : hourly_weather_code,
    'is_day' : hourly_is_day
    
}
hourly_df = pd.DataFrame(hourly_dict)
hourly_df

,date,temperature,weather_code,is_day
0,2025-02-19 15:00:00+00:00,15.560500,3.0,1.0
1,2025-02-19 16:00:00+00:00,15.360500,1.0,1.0
2,2025-02-19 17:00:00+00:00,14.860500,3.0,1.0
3,2025-02-19 18:00:00+00:00,14.310500,3.0,0.0
4,2025-02-19 19:00:00+00:00,12.110500,3.0,0.0
5,2025-02-19 20:00:00+00:00,10.610500,3.0,0.0
6,2025-02-19 21:00:00+00:00,9.910501,3.0,0.0
7,2025-02-19 22:00:00+00:00,8.960501,3.0,0.0
8,2025-02-19 23:00:00+00:00,8.610500,3.0,0.0
9,2025-02-20 00:00:00+00:00,8.660501,3.0,0.0


In [35]:
daily_dict = {
    'weather_code' :  daily_weather_code,
    'temperature_min' : daily_temperature_2m_min,
    'temperature_max' : daily_temperature_2m_max
}

daily_df = pd.DataFrame(daily_dict)
daily_df

,weather_code,temperature_min,temperature_max
0,3.0,2.974,12.774
1,3.0,7.074,15.074
2,51.0,8.580,13.330
3,53.0,7.230,13.030
4,3.0,5.280,6.580
5,NaN,NaN,NaN
6,NaN,NaN,NaN


### test spaCy

### test dateparser

### test geopy